# Deepersense: sim data generator

## imports

In [1]:
import os
# import sys
import rospy
import rosbag
import rospkg
import glob

In [2]:
from simulation_controller import SimulationController
from scene_generator import SceneGenerator, ColorMap
from launchfile_maker import LaunchFileMaker

## file manager

### package path

In [3]:
rospack = rospkg.RosPack()
package_path = rospack.get_path('stonefish_scene_generator')

### iterate files in folder

In [4]:
all_files_path = os.path.join(package_path,'*')
for file_name in glob.glob(all_files_path):
    print(file_name)

/home/zeged/catkin_ws/src/stonefish_scene_generator/launch
/home/zeged/catkin_ws/src/stonefish_scene_generator/src
/home/zeged/catkin_ws/src/stonefish_scene_generator/package.xml
/home/zeged/catkin_ws/src/stonefish_scene_generator/data
/home/zeged/catkin_ws/src/stonefish_scene_generator/CMakeLists.txt


### create data folder

In [5]:
def create_folder(path):
    dir_exists = os.path.isdir(path)
    if not dir_exists:
        os.mkdir(path)
        print('folder created: {}'.format(path))

In [6]:
data_folder_path = os.path.join(package_path, 'data')
create_folder(data_folder_path)

## generate stonefish scene

### make a scene

In [7]:
def make_a_scene(category, main_subject):
    scene = None
    title = category + '_' + main_subject
    seabed_depth = 15
    gen = SceneGenerator(seabed_depth)
    gen.add_empty_frame(ColorMap.jet)
    if main_subject == 'box':
        gen.add_boxes()
    else:
        gen.add_reefs()
    scene = gen.generate()
    return scene

### choose categories

In [8]:
categories = []
categories.append('basic')
# categories.append('multiObect')
shapes = []
shapes.append('box')
shapes.append('sphere')
shapes.append('cylindar')
reefs = ['2019', '2020']
for reef in reefs:
    shapes.append(reef)

test = [categories, shapes]
import itertools
for category, main_subject in list(itertools.product(*test)):
    print(category, main_subject)
    # create category folder
    category_folder_path = os.path.join(data_folder_path, category)
    create_folder(category_folder_path)
    # create main_subject folder
    main_subject_folder_path = os.path.join(category_folder_path, main_subject)
    create_folder(main_subject_folder_path)
    # create a new scene
    scene = make_a_scene(category, main_subject)
    scene_file_full_path = os.path.join(main_subject_folder_path, main_subject)
    print(scene_file_full_path)

('basic', 'box')
/home/zeged/catkin_ws/src/stonefish_scene_generator/data/basic/box/box
('basic', 'sphere')
/home/zeged/catkin_ws/src/stonefish_scene_generator/data/basic/sphere/sphere
('basic', 'cylindar')
/home/zeged/catkin_ws/src/stonefish_scene_generator/data/basic/cylindar/cylindar
('basic', '2019')
/home/zeged/catkin_ws/src/stonefish_scene_generator/data/basic/2019/2019
('basic', '2020')
/home/zeged/catkin_ws/src/stonefish_scene_generator/data/basic/2020/2020


### generate scene

In [9]:
title = 'basic'
seabed_depth = 15
gen = SceneGenerator(seabed_depth)
gen.add_empty_frame(ColorMap.jet)
gen.add_reefs()
scene = gen.generate()
# print(scene)

### save scene to .scn file

In [10]:
def save_scene_to_file(scene, scene_file_full_path):
    f = open(scene_file_full_path + '.scn', "w")
    f.write(scene)
    f.close()

In [11]:
scene_file_full_path = '/home/zeged/catkin_ws/src/stonefish_scene_generator/data/basic/2019/2019.scn'
save_scene_to_file(scene, scene_file_full_path)

## Simulation

### get launch_file_path

In [12]:
launch_file_name = 'auto_generated.launch'
launch_file_path = os.path.join(package_path, 'launch', launch_file_name)
print(launch_file_path)

/home/zeged/catkin_ws/src/stonefish_scene_generator/launch/auto_generated.launch


### edit/make launch file

In [13]:
rate = 300.0
resolution = (1200, 800)
bagfile_path = os.path.join(data_folder_path, title + '.bag')
maker = LaunchFileMaker(scene_file_full_path, rate, resolution, bagfile_path)
maker.save_to_file(launch_file_path)

### setup simulation

In [14]:
controller = SimulationController(launch_file_path)

... logging to /home/zeged/.ros/log/95dd3948-3759-11ec-9e6f-107b448dffed/roslaunch-zeged-System-Product-Name-17374.log


### launch simulation

In [15]:
controller.run()
controller.wait_for_ros()

started roslaunch server http://zeged-System-Product-Name:46331/

SUMMARY

PARAMETERS
 * /rosdistro: melodic
 * /rosversion: 1.14.11

NODES
  /
    rosbag_record_cam (rosbag/record)
    rviz (rviz/rviz)
    stonefish_simulator (stonefish_ros/parsed_simulator)

auto-starting new master
process[master]: started with pid [17435]
ROS_MASTER_URI=http://localhost:11311
setting /run_id to 95dd3948-3759-11ec-9e6f-107b448dffed
process[rosout-1]: started with pid [17454]
started core service [/rosout]
process[stonefish_simulator-2]: started with pid [17457]
process[rviz-3]: started with pid [17462]
process[rosbag_record_cam-4]: started with pid [17463]
[INFO] [1635361845.871930]: started
[INFO] [1635361845.872268]: waiting for topic msg
waiting for message
wait is over
rospy.get_time() - self.timer > 15
(1635361887.03286, 1635361872.03247)
[INFO] [1635361887.033259]: timeout: shutting down
[rosbag_record_cam-4] killing on exit
[rviz-3] killing on exit
[stonefish_simulator-2] killing on exit
[ros

## Bag

### open bag file

In [16]:
import rospy
from collections import defaultdict
import cv2
import numpy as np
from cv_bridge import CvBridge

bag = rosbag.Bag(bagfile, 'r')

## open bag file

In [17]:
bag = rosbag.Bag(bagfile_path, 'r')

## create a folder for images

In [18]:
image_folder_path = bagfile_path.split('.')[0]
print(image_folder_path)
create_folder(image_folder_path)

/home/zeged/catkin_ws/src/stonefish_scene_generator/data/basic


In [19]:
def imshow(win, img):
    cv2.imshow(win, img)
    cv2.waitKey(10)
     

In [20]:
def process_image(msg, topic):
    bridge = CvBridge()
    encoding = 'bgr8'
    img = np.asarray(bridge.imgmsg_to_cv2(msg, encoding))
    return img

In [21]:
def process_depth(msg, topic):
    bridge = CvBridge()
    encoding = '32FC1'
    img = np.asarray(bridge.imgmsg_to_cv2(msg, encoding))
    return img

In [22]:
def normaliaze_depth(msg):
    cv_image_array = np.array(msg, dtype = np.dtype('f8'))
    slice1Copy = cv_image_norm = cv2.normalize(cv_image_array, cv_image_array, 0, 1, cv2.NORM_MINMAX)
    slice1Copy = np.uint8(slice1Copy*255)
    return(slice1Copy)

In [23]:
def save_image(img, filename):
    # print(file_name)
    cv2.imwrite(filename, img)

In [24]:
iterator = bag.read_messages()
counter = defaultdict(lambda: 0)
image_num = 0
for topic, msg, time in iterator:
    msg_time = rospy.Time.to_sec(time) # .to_time(time)
    msg_type = str(type(msg)).split('__')[1].split("'")[0]
    print(msg_time, msg_type, topic)
    counter[topic]+=1
    if msg_type == 'CameraInfo':
        continue
    image_num += 1

    if 'image_color' in topic or 'display' in topic:
        bridged_image = process_image(msg, str(topic))
        imshow(topic, bridged_image)
        file_name = os.path.join(image_folder_path, str(image_num) + '.tif')
        save_image(bridged_image, file_name)



    elif 'image_depth2/image_depth' in topic:
        bridged_image = process_depth(msg, str(topic))
        norm_image = normaliaze_depth(bridged_image)
        imshow(topic, norm_image)
        file_name = os.path.join(image_folder_path, str(image_num) + '.tif')
        save_image(norm_image, file_name)
    else:
        print(topic)

cv2.destroyAllWindows()

(1635361871.9354742, 'Image', '/sparus2/Blue_view_M900_FLS2/display')
(1635361872.0287201, 'CameraInfo', '/image_depth2/camera_info')
(1635361872.0288665, 'CameraInfo', '/camera2/image_raw/camera_info')
(1635361872.0307686, 'Image', '/image_depth2/image_depth')
(1635361872.031991, 'Image', '/camera2/image_raw/image_color')
(1635361872.0865304, 'Image', '/sparus2/Blue_view_M900_FLS2/display')
(1635361872.2027876, 'CameraInfo', '/image_depth2/camera_info')
(1635361872.2040467, 'CameraInfo', '/camera2/image_raw/camera_info')
(1635361872.2048695, 'Image', '/image_depth2/image_depth')
(1635361872.2052114, 'Image', '/sparus2/Blue_view_M900_FLS2/display')
(1635361872.2058969, 'Image', '/camera2/image_raw/image_color')
(1635361872.262145, 'Image', '/sparus2/Blue_view_M900_FLS2/display')
(1635361872.3791416, 'CameraInfo', '/image_depth2/camera_info')
(1635361872.3804283, 'CameraInfo', '/camera2/image_raw/camera_info')
(1635361872.3809626, 'Image', '/image_depth2/image_depth')
(1635361872.381842

In [25]:
print(counter)

defaultdict(<function <lambda> at 0x7f754844ded0>, {'/sparus2/Blue_view_M900_FLS2/display': 150, '/camera2/image_raw/image_color': 104, '/image_depth2/camera_info': 80, '/camera2/image_raw/camera_info': 104, '/image_depth2/image_depth': 80})
